In [1]:
df = spark.sql('select* from titanic')

In [2]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 rows

In [3]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [4]:
df.columns

Out[ 6 ]: 
['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [5]:
df2 = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']) 

In [6]:
df2.show(5)

+--------+------+------+----+-----+-----+-------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
 0| 3| male|22.0| 1| 0| 7.25| S|
 1| 1|female|38.0| 1| 0|71.2833| C|
 1| 3|female|26.0| 0| 0| 7.925| S|
 1| 1|female|35.0| 1| 0| 53.1| S|
 0| 3| male|35.0| 0| 0| 8.05| S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 5 rows

In [7]:
#dropiing the missing data
final_data = df2.na.drop()

In [8]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [9]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex') # transforming into 0,1
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec',dropLast=True) # creating dummay variables

In [10]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex') # transforming into 0,1
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec',dropLast=True) # creating dummay variables

In [11]:
assembler = VectorAssembler(inputCols=['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','SexVec','EmbarkVec','Survived'],
                           outputCol ='features')

In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [13]:
log_reg = LogisticRegression(featuresCol='features',labelCol='Survived')

In [14]:
## creating pipeline
# stages is a list that everything we want to inside the pipeline
pipeline = Pipeline(stages = [gender_indexer,embark_indexer,
                             gender_encoder,embark_encoder,
                             assembler,log_reg])

In [15]:
train,test = final_data.randomSplit([0.7,0.3])

In [16]:
log_reg_model = pipeline.fit(train)

In [17]:
results = log_reg_model.transform(test)
results.show(5)

+--------+------+------+----+-----+-----+------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|Embarked|SexIndex|EmbarkIndex| SexVec| EmbarkVec| features| rawPrediction| probability|prediction|
+--------+------+------+----+-----+-----+------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
 0| 1|female| 2.0| 1| 2|151.55| S| 1.0| 0.0| (1,[],[])|(2,[0],[1.0])|[1.0,2.0,1.0,2.0,...|[13.9623611650607...|[0.99999913657778...| 0.0|
 0| 1| male|19.0| 3| 2| 263.0| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,19.0,3.0,2.0...|[23.5787285797021...|[0.99999999994247...| 0.0|
 0| 1| male|27.0| 0| 2| 211.5| C| 0.0| 1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,27.0,0.0,2.0...|[21.3737063788552...|[0.99999999947818...| 0.0|
 0| 1| male|28.0| 0| 0| 47.1| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,28.0,0.0,0.0...|[18.5107716507659...|[0.99999999086151...| 0.0|
 0| 1| male|29.0| 0| 0| 30.0| S| 0.0| 0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,29.0,0.0,0.0...|[18.4464097961426...|[0.99999999025400...| 0.0|
+--------+------+------+----+-----+-----+------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [20]:
auc = my_eval.evaluate(results)
auc

Out[ 114 ]: 1.0